In [1]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0


In [2]:
import os
import pymatgen
import matplotlib.pyplot as plt
import pandas as pd
from tqdm.notebook import trange, tqdm
from data import get_dichalcogenides_innopolis_202105

In [ ]:
structures = get_dichalcogenides_innopolis_202105()

  0%|          | 0/3480 [00:00<?, ?it/s]

/temporary/pymatgen/pymatgen/io/cif.py:1123: UserWarning:

Issues encountered while parsing CIF: Some fractional co-ordinates rounded to ideal values to avoid issues with finite precision.



In [ ]:
import matplotlib

from megnet.models import MEGNetModel
from megnet.data.graph import GaussianDistance
from megnet.data.crystal import CrystalGraph
from megnet.data.molecule import MolecularGraph
from megnet.utils.preprocessing import StandardScaler
from megnet.callbacks import ModelCheckpointMAE
from pymatgen.core import Lattice, Structure, Molecule

import tensorflow as tf
import numpy as np

In [ ]:
import wandb
from wandb.keras import WandbCallback

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train, test = train_test_split(structures, test_size=0.25, random_state=42)

In [ ]:
wandb.init(project='ai4material_design', entity='kazeev')

In [ ]:
config = wandb.config
config.target = "energy"

In [ ]:
gc = CrystalGraph(bond_converter=GaussianDistance(np.linspace(0, 5, 100), 0.5), cutoff=6)
model = MEGNetModel(nfeat_edge=100, nfeat_global=2, graph_converter=gc, npass=5)

In [ ]:
scaler = StandardScaler.from_training_data(train["initial_structure"],
                                           train[config.target], is_intensive=False)
model.target_scaler = scaler

In [ ]:
model.train(train["initial_structure"], train[config.target],
            validation_structures=test["initial_structure"],
            validation_targets=test[config.target],
            callbacks=[WandbCallback()],
            epochs=100, verbose=1, patience=1000)